In [1]:
!/usr/local/cuda/bin/nvcc –-version


nvcc fatal   : Don't know what to do with '–-version'


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-b7xxrpww
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-b7xxrpww
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=6d28a1c0ff073d5c21888c7ef677a99bf8c2692dd05d2fd8206b73ec88d29760
  Stored in directory: /tmp/pip-ephem-wheel-cache-d63wvtag/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
%%cu
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define m 10

__global__ void mul_r(int *a, int *b, int *c)
{
    int tid = threadIdx.x;
    if (tid < m)
    {
        int sum = 0;
        for (int i = 0; i < m; i++)
        {
            sum += a[i] * b[i * m + tid];
        }
        c[tid] = sum;
    }
}

int main()
{
    int n, c, d, fst[m][m], snd[m][m], t_snd[m][m];
    int row, col, sum_c, a[m], b[m], ans[m];

    // Generate random values for the matrices
    srand(time(0));
    for (c = 0; c < m; c++)
    {
        for (d = 0; d < m; d++)
        {
            fst[c][d] = rand() % 10 + 1;
            snd[c][d] = rand() % 10 + 1;
        }
    }

    printf("Display the elements of the first matrix:\n");
    for (c = 0; c < m; c++)
    {
        for (d = 0; d < m; d++)
        {
            printf("%d\t", fst[c][d]);
        }
        printf("\n");
    }

    printf("Display the elements of the second matrix:\n");
    for (c = 0; c < m; c++)
    {
        for (d = 0; d < m; d++)
        {
            printf("%d\t", snd[c][d]);
        }
        printf("\n");
    }

    // Transpose the second matrix
    for (c = 0; c < m; c++)
    {
        for (d = 0; d < m; d++)
        {
            t_snd[d][c] = snd[c][d];
        }
    }

    printf("\nTranspose of the second matrix:\n");
    for (c = 0; c < m; c++)
    {
        for (d = 0; d < m; d++)
        {
            printf("%d\t", t_snd[c][d]);
        }
        printf("\n");
    }

    // Multiply on CUDA
    int *dev_a, *dev_b, *dev_ans;
    cudaMalloc((void **)&dev_a, m * sizeof(int));
    cudaMalloc((void **)&dev_b, m * m * sizeof(int));
    cudaMalloc((void **)&dev_ans, m * sizeof(int));

    row = 0;
    col = 0;

    cudaEvent_t start, end;
    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);

    for (row = 0; row < m; row++)
    {
        for (d = 0; d < m; d++)
        {
            a[d] = fst[row][d];
        }
        cudaMemcpy(dev_a, a, m * sizeof(int), cudaMemcpyHostToDevice);

        for (col = 0; col < m; col++)
        {
            for (d = 0; d < m; d++)
            {
                b[d] = t_snd[col][d];
                ans[d] = 0;
            }
            cudaMemcpy(dev_b, b, m * sizeof(int), cudaMemcpyHostToDevice);
            cudaMemcpy(dev_ans, ans, m * sizeof(int), cudaMemcpyHostToDevice);

            mul_r<<<1, m>>>(dev_a, dev_b, dev_ans);

            cudaMemcpy(ans, dev_ans, m * sizeof(int), cudaMemcpyDeviceToHost);

            sum_c = 0;
            for (d = 0; d < m; d++)
            {
                sum_c += ans[d];
            }
            snd[row][col] = sum_c;
        }
    }

    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float time = 0;
    cudaEventElapsedTime(&time, start, end);
    printf("Execution time: %f ms\n", time);

    printf("\nMatrix multiplication result:\n");
    for (c = 0; c < m; c++)
    {
        for (d = 0; d < m; d++)
        {
            printf("%d\t", snd[c][d]);
        }
        printf("\n");
    }

    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_ans);

    return 0;
}


Display the elements of the first matrix:
6	10	6	4	7	9	10	9	9	2	
2	2	1	8	2	10	3	9	2	2	
9	10	7	9	10	6	10	10	9	3	
6	1	3	10	3	6	4	4	5	7	
5	7	1	10	7	2	6	3	5	6	
7	3	9	2	10	9	4	5	2	4	
6	3	1	2	4	6	1	2	5	4	
7	2	8	5	10	1	9	9	5	6	
1	1	2	4	2	1	3	10	10	4	
8	9	1	6	1	8	7	3	3	5	
Display the elements of the second matrix:
5	8	1	2	5	1	6	9	1	10	
9	8	2	9	7	10	2	1	2	8	
2	10	8	5	2	3	7	2	9	2	
2	10	1	6	3	3	5	10	4	4	
2	7	8	1	2	4	5	10	5	9	
3	1	7	3	10	7	5	4	4	10	
7	3	8	8	9	2	9	2	6	4	
4	2	6	2	10	7	6	9	10	10	
4	1	5	8	10	8	5	1	6	1	
6	6	9	7	4	1	9	8	7	4	

Transpose of the second matrix:
5	9	2	2	2	3	7	4	4	6	
8	8	10	10	7	1	3	2	1	6	
1	2	8	1	8	7	8	6	5	9	
2	9	5	6	1	3	8	2	8	7	
5	7	2	3	2	10	9	10	10	4	
1	10	3	3	4	7	2	7	8	1	
6	2	7	5	5	5	9	6	5	9	
9	1	2	10	10	4	2	9	1	8	
1	2	9	4	5	4	6	10	6	7	
10	8	2	4	9	10	4	10	1	4	
Execution time: 0.000000 ms

Matrix multiplication result:
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0